# 1. import necessary libraries
In the module "local navigation" of this project, we will need to use several external libraires in order to complete the tache.

In [ ]:
import sys
import time
import cv2
import serial
import math
from threading import Timer
import numpy as np
import matplotlib.pyplot as plt

sys.path.append("localNavigation")
from Thymio import Thymio

sys.path.append("vision")
import vision

# 2. define constants
We define some constants in order to avoid the uncertainty of robot's behavior. ///// A CORRIGER APRES LE FILTER

In [ ]:
#used for defining the position tolerance while moving forward to a coordinate
TOLERENCE_POSITION = 10  
### miau?
ERROR_TOLERENCE=0.1
THRESHOLD_DIST = 2000

# 3. define global variables
We define equally several several global variables which will be useful for local navigation. These variables will be used by different motor commande functions for several times. The variable group "error" is useful for computing PID controller, and the memorised speeds will be useful in case of ANN local avoidance.  

In [ ]:
counter = 0
value_proximity=[0,0,0,0,0,0,0]  # stores horizontal proximity measurements 
value_acceleration=[0,0]         # stores accelerations measurements 
value_speed=[0,0]                # stores motors' speeds measurements 
actual_position=[0,0]            # stores actual position  
actual_angle=0                   # stores actual angle with x axis 
actual_goal=[0,0]                # stores next goal position 
no_detection=False               # stores boolien flag for detection of obstacle.  

# These variables will be used for PID controller 
error_sum=0                      
error = 0
error_prev=0

# These variables will be used for local avoidance (memory)
speed_avoidance_l_prev=0
speed_avoidance_r_prev=0

#  4. Definition of fondamental functions:
- Timer
- Thymio connexion
- sensor measurements
- accessor of measurement variables
- updoad actual position 
There are several functions which should be called in a regular interval of time, such as the filter and the Thymio sampling measurements, because the calculation of them need the intergration. The function "RepeatedTimer" allows to call an arbitary function in a custormized interval. 



In [ ]:
class RepeatedTimer(object):
    """
    definition of timer, which can be called in a fixed interval
    """
    def __init__(self, interval, function, *args, **kwargs):
        self._timer     = None
        self.interval   = interval
        self.function   = function
        self.args       = args
        self.kwargs     = kwargs
        self.is_running = False
        self.start()

    def _run(self):
        self.is_running = False
        self.start()
        self.function(*self.args, **self.kwargs)

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False


In [ ]:
def connexion_thymio():
    """
    This function should be called in order to connect Thymio
    "Thymio is connected :)" will be sent once the connexion is successful
    """
    global th
    th = Thymio.serial(port="COM5", refreshing_rate=0.1)
    time.sleep(3) # To make sure the Thymio has had time to connect
    print("Thymio is connected :)")


In [ ]:
def measure_sensor():
    """
    read the measurements of Thymio. The measurements 
    datas will be stored in global variables
    """
    global value_proximity
    global value_acceleration
    global value_speed

    value_proximity=th['prox.horizontal']
    value_acceleration=th['acc']
    value_speed=[th['motor.left.speed'],th['motor.right.speed']]
    for i in range(2):
        if value_speed[i]>600:
            value_speed[i]=value_speed[i]-2**16
    for i in range(3):
        if value_acceleration[i]>600:
            value_acceleration[i]=value_acceleration[i]-2**16

In [ ]:
def get_sensor_value():
    """
    The accessor function of measurements from this module.
    return: measurements stored in global variables
    """
    return value_proximity,value_acceleration,value_speed


In [ ]:
def get_position(robot_pos):
    """
    This function gets the variable containing the information of position 
    and angle of this robot, unpack this variable to extract and to store 
    these information into the global variables
    
    param: 
    robot_pos :containing position of robot, angle of robot, bool whether robot is detected
    
    return: 
    actual_position,actual_angle : actual position and angle of robot
    """
    
    global actual_angle
    global actual_position
    global no_detection
  
    if robot_pos[2]==True:        # if the robot is detected
        no_detection= False
        actual_position= [robot_pos[0][0],robot_pos[0][1]] # upload actual state of robot
        actual_angle=robot_pos[1]
        return actual_position,actual_angle
    else:                                                  # if robot is not detected
        no_detection=True
        return actual_position,actual_angle                # keep the previous state of robot 

# 5. Motors control functions

In [ ]:
def move(l_speed=500, r_speed=500, verbose=False):
    """
    Sets the motor speeds of the Thymio 
    param l_speed: left motor speed
    param r_speed: right motor speed
    param verbose: whether to print status messages or not
    """
    # Printing the speeds if requested
    if verbose: print("\t\t Setting speed : ", l_speed, r_speed)
    
    # Changing negative values to the expected ones with the bitwise complement
    l_speed = l_speed if l_speed>=0 else 2**16+l_speed
    r_speed = r_speed if r_speed>=0 else 2**16+r_speed

    # Setting the motor speeds
    th.set_var("motor.left.target", l_speed)
    th.set_var("motor.right.target", r_speed)

In [ ]:
def stop(verbose=False):
    """
    param verbose: whether to print status messages or not
    """
    # Printing the speeds if requested
    if verbose:
        print("\t\t Stopping")

    # Setting the motor speeds
    th.set_var("motor.left.target", 0)
    th.set_var("motor.right.target", 0)

# 6. PID controller part:
- compute the error 
- command the motor based on the actualised errors
The error is necessary to PID control. We will want the Thymio to aligne to the direction of actual goal. 

In [1]:
def calculate_error(actual_position,goal,actual_angle):
    """
    This function computes the error between the actual 
    angle and the target angle, we will try to eliminate
    this error with PID controler 
    
    params: 
    actual_position,goal,actual_angle
    
    return: 
    error
    """
    

    global error_sum 
    global no_detection
    global error
    global error_prev
    goal_array = np.array([goal[0],goal[1]])
    actual_position_array = np.array([actual_position[0],actual_position[1]])
   
    direction = goal_array - actual_position_array
    angle = np.arctan2(direction[1], direction[0])
    error_prev = error
    error = -actual_angle + angle 

    if error < -np.pi:
        error += 2*np.pi 
    if error >  np.pi:
        error -= 2*np.pi 

    error_sum += error

    return error

In [ ]:
def follow_the_way_to_dream(actual_position,goal,actual_angle):
    '''
    This function aims to command the motors of Thymio based on
    the angle error previously calculated.
    
    It has 2 parts of controls to the motor: 
        - speed for tracking the way to goal if no obstacle is detected
        - speed of local avoidance if the obstacle is detected
    
    params: 
    actual_position,goal,actual_angle
    '''
   
    global error_sum 
    global value_proximity
    global counter
    global speed_avoidance_l_prev
    global speed_avoidance_r_prev
    '''
    part 1: local avoidance motor speed control
    Thymio will turn in order to avoid the obstacle
    base on the proximity sensor, it uses the memory 
    in order to "memorize" the existance of obstacle. 
    So Thymio can turn more to avoid it.
    
    '''

    if no_detection==False:
        x=np.array([0,0,0,0,0,0,0,0,0])          # array containing the measurement datas and memorized speeds
        sensor_scale = 1500
        
        # ponderation of importance of each sensor contributing the rotation [1:7] 
        # amplitude of movement for each motor due to avoidance [8:9]
        w_l = np.array([40,  20, -20, -20, -40,  30, -10, 8, 0]) 
        w_r = np.array([-40, -20, 20,  20,  40, -10,  30, 0, 8])
        
        x[:7]= np.array(value_proximity) / sensor_scale # compute the roration due to obstacle
        x[7] = speed_avoidance_l_prev / 10 # memory degradation
        x[8] = speed_avoidance_r_prev / 10 

        speed_avoidance_l = np.sum(x * w_l)
        speed_avoidance_r = np.sum(x * w_r)
        speed_avoidance_l_prev=speed_avoidance_l
        speed_avoidance_r_prev=speed_avoidance_r

        #if memory to abstacle avoidance mode is still existing, higher basis speed to pass through
        if x[7] != 0 or x[8] != 0:
            base_speed = 150
        else :            # if no more memory for obstacle avoidance mode, then back to tracking mode, with lower basis speed
            base_speed = 75 


        error = calculate_error(actual_position,goal,actual_angle)
        
        
        # define the parameters for PID controler
        kp = 100
        ki = 3.5
        kd = 8
 
        # anti-winding to avoid a too violent move
        error_sat = 10         

        if error_sum>error_sat:
            error_sum=error_sat
        if error_sum<-error_sat:
            error_sum=-error_sat
        
        # Compute the speed relative to PID controler
        vitesse_PID = kp*error+ki*error_sum + kd *(error-error_prev)

        # combining the final speed 
        speed_l = base_speed + vitesse_PID + speed_avoidance_l
        speed_r = base_speed - vitesse_PID + speed_avoidance_r
        
        move(int(speed_l),int(speed_r))

    else:
        stop()

# 7. Defining the goal and 